In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install peft

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, MistralForCausalLM
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model #prepare_model_for_int8_training
from datasets import load_dataset
import transformers

from huggingface_hub import hf_hub_download

from tqdm import tqdm
import time
import torch

2024-05-29 04:53:13.144394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 04:53:13.144452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 04:53:13.145967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model_name = "LoneStriker/BioMistral-7B-SLERP-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main")


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## DEMO EVAL

In [3]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "trouble digestif?"
prompt=f'''[INST] Explique moi en francais : {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Explique moi en francais : trouble digestif? [/INST] Le trouble digestif désigne un groupe de symptômes ou de maladies qui affectent le fonctionnement du système digestif. Ce système est responsable de la digestion des aliments et de leur transformation en nutriments


In [4]:
transformers.__version__

'4.39.3'

In [5]:
import pandas as pd
valdf = pd.read_csv('/kaggle/input/refomed/refomed_val.csv', sep='\t')
testdf = pd.read_csv('/kaggle/input/refomed/refomed_test.csv', sep='\t')
traindf= pd.read_csv('/kaggle/input/refomed/refomed_train.csv', sep='\t')

In [6]:
prompts = ["Expliquez-moi, en termes simples ou paraphrase ou définition :",
"Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :",
]

queries1 = []
for k in testdf.term:
    queries1.append('[INST]' + prompts[1]+f' {k}'+ '[/INST]')

from datasets import Dataset

testdata = Dataset.from_pandas(pd.DataFrame(queries1, columns=['prompt']))

tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "right"
tokenizer.padding_side = "left" 
def tokenize_function(examples):
    # extract text
    text = examples["prompt"]
    #tokenize and truncate text
    #tokenizer.truncation_side = "right"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="pt",
        #truncation=True,
        #padding=True,
        padding='max_length',
        max_length=512
    )
    return tokenized_inputs

tokenized_testdata = testdata.map(tokenize_function, batched=True)
print(tokenized_testdata)
print(tokenized_testdata[4])

Map:   0%|          | 0/1253 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 1253
})
{'prompt': "[INST]Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition : céphalée d'allure commune[/INST]", 'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [8]:
# Define a collate function to pad inputs dynamically
def collate_fn(batch):
    input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(item['input_ids']) for item in batch], batch_first=True)
    attention_mask = torch.nn.utils.rnn.pad_sequence([torch.tensor(item['attention_mask']) for item in batch], batch_first=True)
    return {'input_ids': input_ids, 'attention_mask': attention_mask}

batch_size = 32  # Define your batch size
from torch.utils.data import DataLoader
data_loader = DataLoader(tokenized_testdata, batch_size=batch_size, collate_fn=collate_fn)
data_loader

model.eval()
device = 'cuda'
for T in [25,50]:
    responses2, times2 = [],[]

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #print(input_ids.shape)
        t1 = time.time()
        outputs = model.generate(input_ids=input_ids, 
                             attention_mask = attention_mask,
                             max_new_tokens=T)
        #print(outputs)
        answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        t2 = time.time()
        responses2.extend(answer)
        times2.extend([(t2-t1)/input_ids.shape[0]]*input_ids.shape[0])
        #break

    pd.DataFrame({"prompts":tokenized_testdata['prompt'], 
                      "zsa": responses2, 
                      "time_taken":times2}).to_csv(f'final_qptq_biom_generation-p4-test-t{T}.csv', sep='\t', index=False)

100%|██████████| 40/40 [50:08<00:00, 75.21s/it]


In [ ]:
model.eval()
device = 'cuda'

for t in [25,50]:
    responses2, times2 = [],[]
    for d in tqdm(tokenized_testdata):
        #print(d)
        t1 = time.time()
        outputs = model.generate(input_ids=torch.tensor([d["input_ids"]]).to("cuda"), 
                             attention_mask = torch.tensor([d["attention_mask"]]).to("cuda"),
                             max_new_tokens=t)
        answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        t2 = time.time()
        responses2.append(answer)
        times2.append(t2-t1)

    pd.DataFrame({"prompts":tokenized_testdata['prompt'], 
                  "zsa": responses2, 
                  "time_taken":times2}).to_csv(f'final_qptq_biom_generation-p4-test-t50.csv', sep='\t', index=False)